In [2]:
from bs4 import BeautifulSoup as bs
import requests
import substring
import pandas as pd
import pymongo

In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Define database and collection
db = client.nfl_teams
collection = db.nfl_teams

In [4]:
url = 'https://www.spotrac.com/nfl/cap/'

In [5]:
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [6]:
print(soup.prettify())

<!DOCTYPE doctype html >
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en"> <![endif]-->
<!--[if IE 9]>    <html class="no-js ie9 oldie" lang="en"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-US">
 <!--<![endif]-->
 <head>
  <!-- start:global -->
  <meta charset="utf-8"/>
  <!-- end:global -->
  <!-- start:page title -->
  <title>
   2018 NFL Team Salary Cap Space Tracker | Spotrac
  </title>
  <!-- end:page title -->
  <!-- start:meta info -->
  <meta content="2018,NFL, salary cap, space, team, rankings" name="keywords">
   <meta content="A real-time look at the 2018 salary cap totals for each NFL team, including estimated cap space." name="description"/>
   <meta content="NFL 2018 Salary Cap Tracker" property="og:title">
    <meta content="Spotrac.com" property="og:site_name"/>
    <meta content="https://www.spotr

In [7]:
test = soup.find_all('span', class_="xs-hide")
print(test)

[<span class="label xs-hide">Viewing:</span>, <span class="xs-hide">Cleveland Browns</span>, <span class="xs-hide">Indianapolis Colts</span>, <span class="xs-hide">San Francisco 49ers</span>, <span class="xs-hide">Tennessee Titans</span>, <span class="xs-hide">Pittsburgh Steelers</span>, <span class="xs-hide">New York Jets</span>, <span class="xs-hide">Houston Texans</span>, <span class="xs-hide">Baltimore Ravens</span>, <span class="xs-hide">Buffalo Bills</span>, <span class="xs-hide">Dallas Cowboys</span>, <span class="xs-hide">Cincinnati Bengals</span>, <span class="xs-hide">Miami Dolphins</span>, <span class="xs-hide">New York Giants</span>, <span class="xs-hide">Jacksonville Jaguars</span>, <span class="xs-hide">Philadelphia Eagles</span>, <span class="xs-hide">Oakland Raiders</span>, <span class="xs-hide">Washington Redskins</span>, <span class="xs-hide">Detroit Lions</span>, <span class="xs-hide">Denver Broncos</span>, <span class="xs-hide">Green Bay Packers</span>, <span class=

In [8]:
teams = []
for x in test:
    name = str(x)
    name = substring.substringByChar(name, startChar='>', endChar='<')
    if name.startswith('>') and name.endswith('<'):
        name = name[1:-1]
    if ":" not in name:
        teams.append(name)
teams

['Cleveland Browns',
 'Indianapolis Colts',
 'San Francisco 49ers',
 'Tennessee Titans',
 'Pittsburgh Steelers',
 'New York Jets',
 'Houston Texans',
 'Baltimore Ravens',
 'Buffalo Bills',
 'Dallas Cowboys',
 'Cincinnati Bengals',
 'Miami Dolphins',
 'New York Giants',
 'Jacksonville Jaguars',
 'Philadelphia Eagles',
 'Oakland Raiders',
 'Washington Redskins',
 'Detroit Lions',
 'Denver Broncos',
 'Green Bay Packers',
 'Chicago Bears',
 'Arizona Cardinals',
 'Atlanta Falcons',
 'New England Patriots',
 'Kansas City Chiefs',
 'Seattle Seahawks',
 'Los Angeles Chargers',
 'Tampa Bay Buccaneers',
 'New Orleans Saints',
 'Los Angeles Rams',
 'Minnesota Vikings',
 'Carolina Panthers']

In [9]:
nfl_data = pd.DataFrame(columns=("Active Players", "Pos", "Base Salary", "Signing Bonus", "Roster Bonus", "Option Bonus", "Workout Bonus", "Restruc Bonus", "Misc", "Dead Cap", "Cap Hit", "Cap %"))
def scraper(team_name):
    formatted = team_name.lower()
    formatted = formatted.replace(" ", "-")
    url = "https://www.spotrac.com/nfl/" + formatted + "/cap"

In [10]:
url = "https://www.spotrac.com/nfl/cleveland-browns/cap/"
team = "Cleveland Browns"
table = pd.read_html(url, match = "Active Players")
team_table = table[0]
team_table.rename(index=str, columns={"Pos.":"Pos", "Restruc. Bonus":"Restruc Bonus", "Misc.":"Misc"}, inplace=True)
team_table.rename(index=str, columns={team_table.iloc[:,0].name:"Active Players"}, inplace=True)
team_table["Team"] = team

In [11]:
nfl_data = pd.concat([nfl_data, team_table])
nfl_data

C:\Users\jgran\Anaconda3\envs\pythondata\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  if __name__ == '__main__':


Active Players  Base Salary  Cap %      Cap Hit       Dead Cap  \
0            Tyrod Taylor  $10,000,000   6.78  $16,000,000  ($16,000,000)   
1           Jarvis Landry   $3,500,000   6.56  $15,500,000  ($34,000,000)   
2           Jamie Collins  $10,650,000   5.25  $12,400,000  ($14,500,000)   
3           Kevin Zeitler  $10,000,000   5.25  $12,400,000  ($23,100,000)   
4           Myles Garrett   $1,847,375   2.93   $6,911,876  ($24,882,754)   
5            J.C. Tretter   $5,000,000   2.86   $6,750,000   ($8,000,000)   
6            Joel Bitonio   $5,500,000   2.54   $6,000,000   ($6,835,223)   
7          Baker Mayfield     $480,000   2.52   $5,942,360  ($32,682,980)   
8           Chris Hubbard   $2,500,000   2.46   $5,800,000   ($9,000,000)   
9             Denzel Ward     $480,000   2.25   $5,302,792  ($29,165,356)   
10          Travis Carrie   $2,000,000   2.24   $5,300,000   ($5,600,000)   
11           Duke Johnson     $957,000   1.65   $3,894,523   ($6,144,523)   
12           Darren Fells   $1,150,000   1.52   $3,600,000   ($4,750,000)   
13            Chris Smith   $1,000,000   1.13   $2,666,666   ($4,600,000)   
14       Britton Colquitt   $2,550,000   1.08   $2,550,000   ($2,550,000)   
15      Terrance Mitchell   $1,500,000   1.04   $2,466,666   ($3,600,000)   
16        Jabrill Peppers     $935,090   1.00   $2,350,451   ($6,586,356)   
17           Drew Stanton   $1,150,000   0.96   $2,265,625   ($4,250,000)   
18            David Njoku     $897,959   0.92   $2,164,794   ($7,793,261)   
19         Emmanuel Ogbah   $1,050,876   0.76   $1,802,629   ($2,103,506)   
20      Damarious Randall   $1,090,381   0.64   $1,514,418              -   
21          Austin Corbet     $480,000   0.58   $1,376,000   ($5,234,718)   
22             Nick Chubb     $480,000   0.57   $1,342,381   ($5,896,309)   
23           Earl Watford     $950,000   0.40     $950,000     ($790,000)   
24         Larry Ogunjobi     $642,500   0.38     $887,891     ($736,173)   
25          Greg Robinson     $790,000   0.33     $790,000     ($790,000)   
26           Joe Schobert     $630,000   0.33     $788,489     ($316,978)   
27        Derrick Kindred     $630,000   0.32     $754,839     ($249,678)   
28            Chad Thomas     $480,000   0.31     $742,235   ($1,048,940)   
29           Seth Devalve     $630,000   0.31     $725,848     ($191,697)   
30             Carl Davis     $720,750   0.31     $720,750              -   
31       Charley Hughlett     $705,000   0.30     $705,000              -   
32       Antonio Callaway     $480,000   0.28     $659,463     ($717,853)   
33   Briean Boddy-Calhoun     $630,000   0.27     $630,000              -   
34        Rashard Higgins     $630,000   0.27     $630,000              -   
35         Anthony Zettel     $630,000   0.27     $630,000              -   
36      Ray-Ray Armstrong     $588,235   0.25     $588,235              -   
37           Trevon Coley     $555,000   0.24     $555,000              -   
38           Genard Avery     $480,000   0.23     $552,236     ($288,947)   
39          Damion Ratley     $480,000   0.22     $528,466     ($193,866)   
40            Brian Price     $518,812   0.22     $518,812              -   
41       Breshad Perriman     $497,647   0.21     $497,647              -   
42       Desmond Harrison     $480,000   0.21     $486,666      ($20,000)   
43        Tavierre Thomas     $480,000   0.20     $480,000              -   
44           D’Juan Hines     $423,529   0.18     $423,529              -   
45            Greg Joseph     $423,529   0.18     $423,529              -   
46      Dontrell Hilliard     $338,820   0.14     $338,820              -   
47         Phillip Gaines     $296,471   0.13     $296,471     ($790,000)   
48          Juston Burris     $296,471   0.13     $296,470              -   
49     Jermaine Whitehead     $261,176   0.11     $261,176              -   
50             Kyle Kalis     $163,235   0.07     $163,235              -   
51  Xav

In [13]:
position_map = pd.read_csv("positions.csv")
new_main = team_table.merge(position_map, on = "Pos")
new_main

Active Players  Pos  Base Salary Signing Bonus Roster Bonus  \
0            Tyrod Taylor   QB  $10,000,000             -   $6,000,000   
1          Baker Mayfield   QB     $480,000    $5,462,360            -   
2            Drew Stanton   QB   $1,150,000    $1,000,000      $15,625   
3           Jarvis Landry   WR   $3,500,000    $1,500,000  $10,500,000   
4        Antonio Callaway   WR     $480,000      $179,463            -   
5         Rashard Higgins   WR     $630,000             -            -   
6           Damion Ratley   WR     $480,000       $48,466            -   
7        Breshad Perriman   WR     $497,647             -            -   
8           Blake Jackson   WR      $28,235             -            -   
9           Jamie Collins  OLB  $10,650,000    $1,250,000     $400,000   
10      Ray-Ray Armstrong  OLB     $588,235             -            -   
11          Kevin Zeitler    G  $10,000,000    $2,400,000            -   
12           Joel Bitonio    G   $5,500,000             -     $500,000   
13          Austin Corbet    G     $480,000      $896,000            -   
14          Myles Garrett   DE   $1,847,375    $5,064,501            -   
15            Chris Smith   DE   $1,000,000    $1,166,666     $400,000   
16         Emmanuel Ogbah   DE   $1,050,876      $751,753            -   
17            Chad Thomas   DE     $480,000      $262,235            -   
18         Anthony Zettel   DE     $630,000             -            -   
19           J.C. Tretter    C   $5,000,000    $1,500,000     $250,000   
20          Chris Hubbard   RT   $2,500,000      $800,000   $2,500,000   
21           Earl Watford   RT     $950,000             -            -   
22            Denzel Ward   CB     $480,000    $4,822,792            -   
23          Travis Carrie   CB   $2,000,000      $900,000   $2,400,000   
24      Terrance Mitchell   CB   $1,500,000      $666,666     $200,000   
25   Briean Boddy-Calhoun   CB     $630,000             -            -   
26         Phillip Gaines   CB     $296,471             -            -   
27          Juston Burris   CB     $296,471             -            -   
28           Duke Johnson   RB     $957,000      $927,523   $2,000,000   
29             Nick Chubb   RB     $480,000      $862,381            -   
30      Dontrell Hilliard   RB     $338,820             -            -   
31           Darren Fells   TE   $1,150,000      $700,000   $1,650,000   
32            David Njoku   TE     $897,959    $1,266,835            -   
33           Seth Devalve   TE     $630,000       $95,848            -   
34       Britton Colquitt    P   $2,550,000             -            -   
35        Jabrill Peppers   SS     $935,090    $1,415,361            -   
36      Damarious Randall   FS   $1,090,381             -     $424,037   
37     Jermaine Whitehead   FS     $261,176             -            -   
38         Larry Ogunjobi   DT     $642,500      $245,391            -   
39             Carl Davis   DT     $720,750             -            -   
40           Trevon Coley   DT     $555,000             -            -   
41            Brian Price   DT     $518,812             -            -   
42          Greg Robinson   LT     $790,000             -            -   
43       Desmond Harrison   LT     $480,000        $6,666            -   
44           Joe Schobert  ILB     $630,000      $158,489            -   
45           Genard Avery  ILB     $480,000       $72,236            -   
46  Xavier Woodson-Luster  ILB      $97,941             -            -   
47        Derrick Kindred    S     $630,000      $124,839            -   
48        Tavierre Thomas    S     $480,000             -            -   
49       Charley Hughlett   LS     $705,000             -            -   
50           D’Juan Hines   LB     $423,529             -            -   
51            Greg Joseph    K     $423,529             -            -   
52             Kyle Kalis    T     $163,235             -            -   

   Option Bonu

In [ ]:
db.collection.insert_many(new_main.to_dict("records"))

In [58]:
side = new_main[["Pos Side", "Cap Hit", "Team"]]
side["Cap Hit"] = side["Cap Hit"].str.replace('$','')
side["Cap Hit"] = side["Cap Hit"].str.replace(',','')
side.head()
side["Cap Hit"] = pd.to_numeric(side["Cap Hit"])
side.groupby("Pos Side").mean()

C:\Users\jgran\Anaconda3\envs\pythondata\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\jgran\Anaconda3\envs\pythondata\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\jgran\Anaconda3\envs\pythondata\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

Cap Hit
Pos Side                   
Defense        1.920753e+06
Offense        3.701419e+06
Special Teams  1.226176e+06

In [52]:
cat = new_main[["Pos Category", "Cap Hit", "Team"]]
cat["Cap Hit"] = cat["Cap Hit"].str.replace('$','')
cat["Cap Hit"] = cat["Cap Hit"].str.replace(',','')
cat.head()
cat["Cap Hit"] = pd.to_numeric(side["Cap Hit"])
cat.groupby("Pos Category").mean()

C:\Users\jgran\Anaconda3\envs\pythondata\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\jgran\Anaconda3\envs\pythondata\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\jgran\Anaconda3\envs\pythondata\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

Cap Hit
Pos Category                
Defensive Line  1.715095e+06
Linebacker      2.475072e+06
Offensive Line  3.857322e+06
Passing         4.045203e+06
Rushing         1.858575e+06
Secondary       1.786662e+06
Special Teams   1.226176e+06

In [55]:
# list of position names
position_cats = position_map["Pos Category"]
position_cats = position_cats.unique()
position_list = position_cats.tolist()
position_list

['Offensive Line',
 'Secondary',
 'Defensive Line',
 'Rushing',
 'Linebacker',
 'Special Teams',
 'Passing']

In [56]:
# loop through list
for x in position_list:
    
    # filter df on list
    cat_filtered = cat[cat['Pos Category'] == x]
    
    # group by team.sum()
    cat_final = cat_filtered.groupby("Team").sum()
    
    # average the column
    print(x, cat_final["Cap Hit"].mean())

Offensive Line 34715901.0
Secondary 19653283.0
Defensive Line 15435859.0
Rushing 5575724.0
Linebacker 14850430.0
Special Teams 3678529.0
Passing 48542438.0


In [62]:
# list of position names
position_side = position_map["Pos Side"]
position_side = position_side.unique()
side_list = position_side.tolist()
print(side_list)

# loop through list
for x in side_list:
    
    # filter df on list
    side_filtered = side[side['Pos Side'] == x]
    
    # group by team.sum()
    side_final = side_filtered.groupby("Team").sum()
    
    # average the column
    print(x, side_final["Cap Hit"].mean())

['Offense', 'Defense', 'Special Teams']
Offense 88834063.0
Defense 49939572.0
Special Teams 3678529.0
